In [ ]:
#r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
using  NSE_SIMPLE.LowMach;
Init();

Error: (30,8): error CS0246: The type or namespace name 'NSE_SIMPLE' could not be found (are you missing a using directive or an assembly reference?)

In [ ]:
BoSSSshell.WorkflowMgm.Init("HeatedBackwardFacingStep");

Project name is set to 'HeatedBackwardFacingStep'.
NotSupportedException caught during creation/opening of default database: `AllowedDatabasesPaths` not specified, unable to create Database (should be specified in ~/.BoSSS/etc/BatchProcessorConfig.json)..


In [ ]:
// static var myBatch = GetDefaultQueue();

In [ ]:
// static var myDb = myBatch.CreateOrOpenCompatibleDatabase("HeatedBackwardFacingStep");
// static var myDb = OpenOrCreateDatabase(@"C:\Databases\HeatedBackwardFacingStep2");
static var myBatch = BoSSSshell.ExecutionQueues[2];

static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\HeatedBackwardFacingStep_noHeat");

In [ ]:
BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

Opening existing database '\\hpccluster\hpccluster-scratch\gutierrez\HeatedBackwardFacingStep_noHeat'.


## Create grid

In [ ]:
int Resolution = 5;
int dg = 2;
double Reynolds = 200;
bool HeatOn = false;
int MPI_PROCS = 8;
static double Hdim = 30; // mm channel height
static double H = 30 / Hdim;// nondim channel height
static double S = 15 / Hdim;  //nondim step height
static double L0 = S; // nondim upstream wall
static double L = 40 * S; // downstream wall
//double L = 50 * S; // downstream wall

static int xNodesMultiplier = 7;
double h = Math.Pow(2, -Resolution + 1); // cell length
double cells = 1 / h;
int cells2 = (int)cells;


In [ ]:
vel

0.20693333333333333

In [ ]:
public static class GridFactory {

    public static double[] GetXNodes_LinspaceOneSide(int Res) {
        // left part
        var _leftNodes = GenericBlas.SinLinSpacing(-2 * L0, 0, 0.9, (1 * Res) + 1);
        var leftNodes = _leftNodes.GetSubVector(_leftNodes.Length / 2, _leftNodes.Length / 2 + 1);
        //right part
        var _rightNodes = GenericBlas.SinLinSpacing(0, 2 * L, 0.9, (xNodesMultiplier * Res) + 1);
        var rightNodes = _rightNodes.GetSubVector(0, _rightNodes.Length / 2 + 1);

        List<double> listX = new List<double>();
        listX.AddRange(leftNodes.Take(leftNodes.Count() - 1).ToList());
        listX.AddRange(rightNodes.Take(rightNodes.Count() + 1).ToList());

        return listX.ToArray();
    }

    private static double[] GetYNodes_LinspaceOneSide(int Res) {
        //Bottom part
        var _bottomNodes = GenericBlas.SinLinSpacing(-S, S, 0.9, (2 * Res) + 1);
        var bottomNodes = _bottomNodes.GetSubVector(_bottomNodes.Length / 2, _bottomNodes.Length / 2 + 1);
        //upper part
        var _upperNodes = GenericBlas.SinLinSpacing(S, 3 * S, 0.9, (2 * Res) + 1);
        var upperNodes = _upperNodes.GetSubVector(0, _upperNodes.Length / 2 + 1);
        List<double> listY = new List<double>();
        listY.AddRange(bottomNodes.Take(bottomNodes.Count() - 1).ToList());
        listY.AddRange(upperNodes.Take(upperNodes.Count() + 1).ToList());

        return listY.ToArray();
    }

    public static double[] GetXNodes_LinspaceBothSides(int Res) {
        // left part
        var _leftNodes = GenericBlas.Linspace(-L0, 0,  (1 * Res) + 1);
        //right part
        var _rightNodes = GenericBlas.SinLinSpacing(0, L, 0.9, (15 * Res) + 1);

        List<double> listX = new List<double>();
        listX.AddRange(_leftNodes.Take(_leftNodes.Count() - 1).ToList());
        listX.AddRange(_rightNodes.Take(_rightNodes.Count() + 1).ToList());

        return listX.ToArray();
    }

    private static double[] GetYNodes_LinspaceBothSides(int Res) {
        //Bottom part
        var _bottomNodes = GenericBlas.SinLinSpacing(0, S, 0.8, (3 * Res) + 1);
        //upper part
        var _upperNodes = GenericBlas.Linspace(S, 2 * S, (2 * Res) + 1);
        List<double> listY = new List<double>();
        listY.AddRange(_bottomNodes.Take(_bottomNodes.Count() - 1).ToList());
        listY.AddRange(_upperNodes.Take(_upperNodes.Count() + 1).ToList());

        return listY.ToArray();
    }

    public static Grid2D GenerateGrid(int Res) {
        // Grid2D grd;

        double[] CutOut1Point1 = new double[2] { 0.0, 0.0 };
        double[] CutOut1Point2 = new double[2] { -L0, S };

        var CutOut1 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut1.AddPoint(CutOut1Point1);
        CutOut1.AddPoint(CutOut1Point2);

        // if (LinspaceBothSidesOK) {
        //     var xNodes = GetXNodes_LinspaceBothSides(Res);
        //     var yNodes = GetYNodes_LinspaceBothSides(Res);
        //     grd = Grid2D.Cartesian2DGrid(xNodes, yNodes, CutOuts: CutOut1);
        // } else {
        //     var xNodes = GetXNodes_LinspaceOneSide(Res);
        //     var yNodes = GetYNodes_LinspaceOneSide(Res);
        //     grd = Grid2D.Cartesian2DGrid(xNodes, yNodes, CutOuts: CutOut1);
        // }
        var xNodes = GetXNodes_LinspaceOneSide(Res);
        var yNodes = GetYNodes_LinspaceBothSides(Res);
         var grd    = Grid2D.Cartesian2DGrid(xNodes, yNodes, CutOuts: CutOut1);
        grd.EdgeTagNames.Add(1, "Velocity_Inlet");
        grd.EdgeTagNames.Add(3, "Pressure_Outlet");
        grd.EdgeTagNames.Add(2, "NoSlipNeumann");
        grd.EdgeTagNames.Add(4, "Wall");

        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            //Inlet oxidizer
            if (Math.Abs(x + L0) < 1e-8)
                return 1;

            //Outlet
            if (Math.Abs(x - L) < 1e-8)
                return 3;

            //Bottom Wall
            if (Math.Abs(y - 0) < 1e-8 && x > 1e-8)
                return 4;

            return 2;
        });

        myDb.SaveGrid(ref grd);

        return grd;
    }
}

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double const_val, double S, double umean ) {
        using(var stw = new System.IO.StringWriter()) {
           
           stw.WriteLine("static class BoundaryValues {");


           stw.WriteLine("  static public double ConstantVal(double[] X) {");
           stw.WriteLine("    return " + const_val+";");
           stw.WriteLine("  }");

           
           stw.WriteLine("  static public double ParabolaVelocity(double[] X) {");
           stw.WriteLine("    return  -6 * "+umean+" * (X[1] -"+ S+") * (X[1] - 2 * "+S+") / ("+S * S+") ;");
           stw.WriteLine("  }");


           stw.WriteLine("}"); 
           return stw.ToString();
        }
    }
    static public Formula Get_ConstantVal(double const_val, double S, double umean){
        return new Formula("BoundaryValues.ConstantVal", AdditionalPrefixCode:GetPrefixCode(const_val,S,umean));
    }

    static public Formula Get_ParabolaVelocity(double const_val, double S, double umean){
        return new Formula("BoundaryValues.ParabolaVelocity", AdditionalPrefixCode:GetPrefixCode(const_val,S,umean));
    }

}

## Send and run jobs
Calculations of the SIMPLE code

In [ ]:
var controls = new List<LowMachSIMPLEControl>();

In [ ]:

 static LowMachSIMPLEControl SteadyCouetteFlowWithTemperatureGradient() {
    LowMachSIMPLEControl c = new LowMachSIMPLEControl();

    c.DbPath = @"NUnitTests.zip";
    c.savetodb = false;

    c.GridGuid = new Guid("b3eb0eac-d1a1-440c-9f08-5dae1284607d");
    c.GridPartType = GridPartType.METIS;

    c.ProjectName = "Couette with temperature gradient";
    c.ProjectDescription = "Steady Low Mach SIMPLE";

    // Required fields
    c.FieldOptions.Add(
        VariableNames.VelocityX,
        new FieldOpts() { Degree = 2, SaveToDB = FieldOpts.SaveToDBOpt.TRUE });
    c.FieldOptions.Add(
        VariableNames.VelocityY,
        new FieldOpts() { Degree = 2, SaveToDB = FieldOpts.SaveToDBOpt.TRUE });
    c.FieldOptions.Add(
        VariableNames.Pressure,
        new FieldOpts() { Degree = 1, SaveToDB = FieldOpts.SaveToDBOpt.TRUE });
    c.FieldOptions.Add(
        VariableNames.Temperature,
        new FieldOpts() { Degree = 2, SaveToDB = FieldOpts.SaveToDBOpt.TRUE });
    c.FieldOptions.Add(
        VariableNames.ThermodynamicPressure,
        new FieldOpts() { Degree = 0, SaveToDB = FieldOpts.SaveToDBOpt.TRUE });

    c.FieldOptions.Add(
        "Density",
        new FieldOpts() { Degree = 2, SaveToDB = FieldOpts.SaveToDBOpt.TRUE });
    c.FieldOptions.Add(
        "Eta",
        new FieldOpts() { Degree = 2, SaveToDB = FieldOpts.SaveToDBOpt.TRUE });

    // Auxiliary fields
    c.FieldOptions.Add(
        "DivB4",
        new FieldOpts() { Degree = 1, SaveToDB = FieldOpts.SaveToDBOpt.FALSE });
    c.FieldOptions.Add(
        "DivAfter",
        new FieldOpts() { Degree = 1, SaveToDB = FieldOpts.SaveToDBOpt.FALSE });
    c.FieldOptions.Add(
        "OperatorTest_x",
        new FieldOpts() { Degree = 2, SaveToDB = FieldOpts.SaveToDBOpt.FALSE });
    c.FieldOptions.Add(
        "OperatorTest_y",
        new FieldOpts() { Degree = 2, SaveToDB = FieldOpts.SaveToDBOpt.FALSE });
    c.FieldOptions.Add(
        "OperatorAna_x",
        new FieldOpts() { Degree = 2, SaveToDB = FieldOpts.SaveToDBOpt.FALSE });
    c.FieldOptions.Add(
        "OperatorAna_y",
        new FieldOpts() { Degree = 2, SaveToDB = FieldOpts.SaveToDBOpt.FALSE });

    c.InitialValues_Evaluators.Add(VariableNames.VelocityX, X => X[1]);
    c.InitialValues_Evaluators.Add(VariableNames.VelocityY, X => 0.0);
    c.InitialValues_Evaluators.Add(VariableNames.Pressure, X => -0.97809076838538383654 * Log(1.2 * X[1] + 0.4) - 0.06641065188714375468);
    c.InitialValues_Evaluators.Add(VariableNames.Temperature, X => (1.6 - 0.4) * X[1] + 0.4);

    c.Algorithm = SolutionAlgorithms.Steady_SIMPLE;
    c.NoOfTimesteps = 1;
    c.L2NormPressureCorrection = 1.0e-6;
    c.L2NormVelocityResidual = 1.0e-6;
    c.L2NormTemperatureResidual = 1e-6;

    c.PredictorSolverFactory = () => new PARDISOSolver();
    c.CorrectorSolverFactory = () => new PARDISOSolver();
    c.TemperatureSolverFactory = () => new PARDISOSolver();

    c.AnalyticVelocityX = X => -0.33333333333333333332 + 1.2523108062960316903 * (X[1] + 0.11013981986815616002).Pow(3.0 / 5.0);
    c.AnalyticVelocityY = X => 0.0;
    c.AnalyticPressure = X => -1.4882576522041879973 * (1.9716158024185796207 * X[1] + 0.21715340932759252572).Pow(2.0 / 5.0) + 1.5508248735452649660;
    c.AnalyticTemperature = X => (1.9716158024185796207 * X[1] + 0.21715340932759252572).Pow(3.0 / 5.0);

    c.AddBoundaryValue("velocity_inlet_top", VariableNames.VelocityX, X => 1.0);
    c.AddBoundaryValue("velocity_inlet_top", VariableNames.VelocityY, X => 0.0);
    c.AddBoundaryValue("velocity_inlet_top", VariableNames.Temperature, X => 1.6);
    c.AddBoundaryValue("velocity_inlet_left", VariableNames.VelocityX, c.AnalyticVelocityX);
    c.AddBoundaryValue("velocity_inlet_left", VariableNames.VelocityY, c.AnalyticVelocityY);
    c.AddBoundaryValue("velocity_inlet_left", VariableNames.Temperature, c.AnalyticTemperature);
    c.AddBoundaryValue("velocity_inlet_right", VariableNames.VelocityX, c.AnalyticVelocityX);
    c.AddBoundaryValue("velocity_inlet_right", VariableNames.VelocityY, c.AnalyticVelocityY);
    c.AddBoundaryValue("velocity_inlet_right", VariableNames.Temperature, c.AnalyticTemperature);
    c.AddBoundaryValue("wall_bottom", VariableNames.Temperature, X => 0.4);

    c.PhysicsMode = PhysicsMode.LowMach;
    c.ThermodynamicPressureMode = ThermodynamicPressureMode.Constant;
    c.Reynolds = 10.0;
    c.Prandtl = 0.71;
    c.Gamma = 1.4;
    c.EoS = new MaterialLawLowMach(600.0, MaterialParamsMode.PowerLaw, false,false,c.Prandtl);

    c.Froude = 0.92303846;
    c.GravityDirection = new double[] { 0.0, -1.0 };

    c.PressureReferencePoint = new double[] { 0.0, 0.5 };
    c.PressureMeanValue = 0.0;

    c.PredictorApproximation = PredictorApproximations.BlockDiagonal;
    c.PressureStabilizationScaling = 0.0;
    c.PredictorApproximationUpdateCycle = 1;
    c.MaxNoSIMPLEsteps = 1000;
    c.SavePeriodSIMPLE = 1000;
    c.RelaxationFactorPressure = 0.5;
    c.RelexationFactorVelocity = 0.8;
    c.RelexationFactorTemperature = 1.0;
    c.RelaxationModeTemperature = RelaxationTypes.Implicit;
    c.ViscousPenaltyScaling = 1.0;
    c.MaxFlowSolverIterations = 1;
    c.MaxTemperatureSolverIterations = 1;
    c.PrintLinerSolverResults = false;

    int queryQuadOrder = 15;
    c.Queries.Add(
        "SolL2err_u",
        QueryLibrary.L2Error(VariableNames.VelocityX, c.AnalyticVelocityX, queryQuadOrder));
    c.Queries.Add(
        "SolL2err_v",
        QueryLibrary.L2Error(VariableNames.VelocityY, c.AnalyticVelocityY, queryQuadOrder));
    c.Queries.Add(
        "SolL2err_p",
        QueryLibrary.L2Error(VariableNames.Pressure, c.AnalyticPressure, queryQuadOrder));
    c.Queries.Add(
        "SolL2err_T",
        QueryLibrary.L2Error(VariableNames.Temperature, c.AnalyticTemperature, queryQuadOrder));

    return c;
}
    


## Run Simulations

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);
var C = GiveMeTheCtrlFile(dg, cells2, Reynolds, false); 
string jobName       = C.SessionName;
Console.WriteLine(jobName);
var oneJob           = new Job(jobName, solver);
oneJob.SetControlObject(C);
oneJob.NumberOfMPIProcs = MPI_PROCS;
oneJob.NumberOfThreads = 1;

oneJob.Activate(myBatch); 


////////////////////////////////////////////////////////////////////////////////////
BackwardFacingStep
////////////////////////////////////////////////////////////////////////////////////


Grid Edge Tags changed.
Heated_BackwardFacingStepdg2k16Re200EnergyOK0
Deploying job Heated_BackwardFacingStepdg2k16Re200EnergyOK0 ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\gutierrez\binaries\HeatedBackwardFacingStep-XNSEC2022Feb16_165028
copied 55 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.



In [ ]:
// Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);
// var C = GiveMeTheCtrlFile(dg, cells2, Reynolds, false); 
// string jobName       = C.SessionName;
// Console.WriteLine(jobName);
// var oneJob           = new Job(jobName, solver);
// oneJob.SetControlObject(C);
// oneJob.NumberOfMPIProcs = MPI_PROCS;

// oneJob.Activate(myBatch); 


In [ ]:
oneJob.ShowOutput()

Starting external console ...
(You may close the new window at any time, the job will continue.)


In [ ]:
// // // wait for all jobs to finish (up to 1 day, check every 1 minutes)
// BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*1), PollingIntervallSeconds:(60*1));

In [ ]:
// // detect failed Jobs in the job management
// var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
//     .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
//                   || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
// suspects

In [ ]:
// NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");